<a href="https://colab.research.google.com/github/jjbmsda/ThesisStudy/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DmmlTiSV-master

/content/drive/MyDrive/DmmlTiSV-master


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import io
import requests

from PIL import Image
from torchvision import models
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
#import cv2
import pdb
import torch
import torch.nn as nn
import glob
import random
import numpy as np
import time
import math
import sys
import scipy.io as sio
from sklearn import *
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils import data
from tqdm import tqdm
from config import Config
from sklearn import *
from sklearn import metrics as Metrics
from models import resnet
from torch.nn import DataParallel

import librosa
from scipy.signal.windows import hamming
import soundfile as sf

SAMPLE_RATE = 16000
FEATURE = 'fft'#
FEATURE_LEN = 161#
WIN_LEN = 0.02#
WIN_STEP = 0.01#

N_FFT = int(WIN_LEN * SAMPLE_RATE)#
HOP_LEN = int(WIN_STEP * SAMPLE_RATE)#

N_FRAMES = 300#
DURATION = (N_FRAMES - 1) * WIN_STEP#
N_SAMPLES = int(DURATION * SAMPLE_RATE)#

N_TEST_FRAMES = 300#
TEST_DURATION = (N_TEST_FRAMES - 1) * WIN_STEP#
N_TEST_SAMPLES = int(TEST_DURATION * SAMPLE_RATE)#


def load_audio(filename, start=0, stop=None, resample=True):#
    sr = SAMPLE_RATE
    y, sr = sf.read(filename, start=start, stop=stop, dtype='float32', always_2d=True)
    y = np.squeeze(y)
    return y, sr


class my_dataset(data.Dataset):
    def __init__(self, root, lists):
        self.root = root
        self.lists = lists

        self.transforms = transforms.Compose([
                transforms.ToTensor(),])

    def __getitem__(self, index):
        im_pth = self.lists[index]
        im_pth = os.path.join(self.root, im_pth)
        im, sr = load_audio(im_pth)
        S = librosa.core.stft(im, n_fft=N_FFT, hop_length=HOP_LEN, window=hamming)#
        feature, _ = librosa.magphase(S)
        npy = np.log1p(feature)#
        npy = npy.transpose()
        npy = npy[np.newaxis, :, :]
        im = np.repeat(npy, 3, axis=0)
        #print(im.shape)



        num_frame = 300
        l = im.shape[1]
        ims = []
        for i in range(10):
            
            if l <= num_frame:
                new = np.zeros((3, num_frame, 161))
                new[:, :l, :] = im
                new[:, num_frame-l:, :] = im[:, :l, :]
                npy = new
            else:
                randint = np.random.randint(l - num_frame)
                npy = im[:, randint: randint+num_frame, :]
            #print(npy.shape)



            npy = np.swapaxes(npy,1,2)
            mu = np.average(npy)
            sigma = np.std(npy)
            npy = (npy - mu) / max(sigma, 0.001)
            #print(npy.shape)
            npy = self.transforms(npy).float()
            npy = npy.permute(1,2,0)
            npy = npy.unsqueeze(0)
            #print(npy.shape)
            if i == 0:
                ims = npy
            else:
                ims = torch.cat((ims, npy), 0)

        #print(ims.shape)


        return ims, self.lists[index]

    def __len__(self):
        return len(self.lists)






#root_dir = "/data2/xjw/vox1_test_npy/"
#test_path = "/home/xjw/workspace/metric_learning_speaker_verification/veri_test.txt"
#path = "/home/xjw/workspace/triplet-network-pytorch-master/veri_png.txt"

def get_lists(path):
    files= []
    f = open(path, "r")
    lines = f.readlines()
    for line in lines:
        line = line.split()
        #print(line[1], line[2])
        if line[1] not in files:
            files.append(line[1])
        if line[2] not in files:
            files.append(line[2])
    return files

def get_featurs(model, root_dir, lists):

    #pbar = tqdm(total=len(lists))
    #for idx, img_path in enumerate(lists):
    #    pbar.update(1)
    features_dict = {}

    trainloader = my_dataset(root=root_dir, lists=lists)
    dl = data.DataLoader(trainloader, batch_size=1)

    #trainloader = data.DataLoader(lists, batch_size=1)
    for i, (img, im_pth) in enumerate(dl):
        if i % 100 == 0:
            print(i)
        img = img.cuda()
        img = torch.squeeze(img)
        #img = img.unsqueeze(1)
        #print(i, img.shape, im_pth[0])
        if i == 0:
            _, feature = model(img)
            feature = feature.detach().cpu().numpy()
            features_dict[im_pth[0]] = feature 
            features = feature
        else:
            _, feature = model(img)
            feature = feature.detach().cpu().numpy()
            features_dict[im_pth[0]] = feature 
            features = np.concatenate((features, feature), axis=0)
    return features, features_dict




def cosin_metric(features1, features2):
    score = np.mean(np.matmul(features1, features2.T))
    #print("score:", score)
    return score



#score = score_vector
#label = target_label_vector


def calculate_eer_auc_ap(label,distance):

    fpr, tpr, thresholds = Metrics.roc_curve(label, distance, pos_label=1)
    AUC = Metrics.roc_auc_score(label, distance, average='macro', sample_weight=None)
    AP = Metrics.average_precision_score(label, distance, average='macro', sample_weight=None)

    # Calculating EER
    intersect_x = fpr[np.abs(fpr - (1 - tpr)).argmin(0)]
    EER = intersect_x
    miss = 1 - fpr
    false = 1 - tpr
    #false = 1 - fpr
    #miss = 1 - tpr
    #C_det = 0.01*

    return EER,AUC,AP,fpr, tpr, miss, false

# K-fold validation for ROC



def vox_test(model, test_root, test_list):

    lists = get_lists(test_list)
    print(len(lists))

    f = open(test_list, "r")
    lines = f.readlines()
    score_vector = np.zeros((len(lines), 1))
    target_label_vector = np.zeros((len(lines), 1))
    features, features_dict = get_featurs(model, test_root, lists)

    lists = np.array(lists)

    scores = []
    labels = []

    for i, line in enumerate(lines):
        line = line.split()
        labels.append(int(line[0]))
        score = cosin_metric(features_dict[line[1]], features_dict[line[2]])
        #print(score)
        scores.append(score)    


    score = np.array(scores)[:, np.newaxis]
    label = np.array(labels)[:, np.newaxis]

#score_vector = softmax.softmax(score_vector)

#np.save(os.path.join('/home/xjw/workspace/metric_learning_speaker_verification/result/','score_vector.npy'),score)
#np.save(os.path.join('/home/xjw/workspace/metric_learning_speaker_verification/result/','target_label_vector.npy'),label)
    k = 1
    step = int(label.shape[0] / float(k))
    EER_VECTOR = np.zeros((k,1))
    AUC_VECTOR = np.zeros((k,1))
    C_det = np.zeros((k,1))
    for split_num in range(k):
        index_start = split_num * step
        index_end = (split_num + 1) * step
    #print(label[index_start:index_end])
    #print(score[index_start:index_end])
        EER_temp,AUC_temp,AP,fpr, tpr, miss, false = calculate_eer_auc_ap(label[index_start:index_end],score[index_start:index_end])
        EER_VECTOR[split_num] = EER_temp * 100
        AUC_VECTOR[split_num] = AUC_temp * 100
        C_det[split_num] = 10 *0.01* np.mean(miss) + 1 * 0.99 * np.mean(false)

    eer = np.mean(EER_VECTOR)
    print("eer:", eer)
    return eer





if __name__ == '__main__':

    opt = Config()
    #model = resnet34()


    logs_root = opt.logs_root
    logs = os.listdir(logs_root)

    model = resnet.resnet50(pretrained=False,num_classes=5994).cuda()
    model = nn.DataParallel(model)
    model.cuda()
    #model.load_state_dict(torch.load("./logs/model_avgpool2.pth.tar"))

    model.eval()
    vox_test(model, opt.vox_root, opt.vox_test_list)

FileNotFoundError: ignored

In [ ]:
def mfcc(signal,samplerate=16000,winlen=0.025,winstep=0.01,numcep=13,
         nfilt=26,nfft=None,lowfreq=0,highfreq=None,preemph=0.97,ceplifter=22,appendEnergy=True,
         winfunc=lambda x:numpy.ones((x,))):
    """Compute MFCC features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param numcep: the number of cepstrum to return, default 13
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is None, which uses the calculate_nfft function to choose the smallest size that does not drop sample data.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param ceplifter: apply a lifter to final cepstral coefficients. 0 is no lifter. Default is 22.
    :param appendEnergy: if this is true, the zeroth cepstral coefficient is replaced with the log of the total frame energy.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: A numpy array of size (NUMFRAMES by numcep) containing features. Each row holds 1 feature vector.
    """
    nfft = nfft or calculate_nfft(samplerate, winlen)
    feat,energy = fbank(signal,samplerate,winlen,winstep,nfilt,nfft,lowfreq,highfreq,preemph,winfunc)
    feat = numpy.log(feat)
    feat = dct(feat, type=2, axis=1, norm='ortho')[:,:numcep]
    feat = lifter(feat,ceplifter)
    if appendEnergy: feat[:,0] = numpy.log(energy) # replace first cepstral coefficient with log of frame energy
    return feat

In [ ]:

# calculate filterbank features. Provides e.g. fbank and mfcc features for use in ASR applications
# Author: James Lyons 2012
from __future__ import division
import numpy
from python_speech_features import sigproc
from scipy.fftpack import dct

def calculate_nfft(samplerate, winlen):
    """Calculates the FFT size as a power of two greater than or equal to
    the number of samples in a single window length.
    
    Having an FFT less than the window length loses precision by dropping
    many of the samples; a longer FFT than the window allows zero-padding
    of the FFT buffer which is neutral in terms of frequency domain conversion.
    :param samplerate: The sample rate of the signal we are working with, in Hz.
    :param winlen: The length of the analysis window in seconds.
    """
    window_length_samples = winlen * samplerate
    nfft = 1
    while nfft < window_length_samples:
        nfft *= 2
    return nfft

def mfcc(signal,samplerate=16000,winlen=0.025,winstep=0.01,numcep=13,
         nfilt=26,nfft=None,lowfreq=0,highfreq=None,preemph=0.97,ceplifter=22,appendEnergy=True,
         winfunc=lambda x:numpy.ones((x,))):
    """Compute MFCC features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param numcep: the number of cepstrum to return, default 13
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is None, which uses the calculate_nfft function to choose the smallest size that does not drop sample data.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param ceplifter: apply a lifter to final cepstral coefficients. 0 is no lifter. Default is 22.
    :param appendEnergy: if this is true, the zeroth cepstral coefficient is replaced with the log of the total frame energy.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: A numpy array of size (NUMFRAMES by numcep) containing features. Each row holds 1 feature vector.
    """
    nfft = nfft or calculate_nfft(samplerate, winlen)
    feat,energy = fbank(signal,samplerate,winlen,winstep,nfilt,nfft,lowfreq,highfreq,preemph,winfunc)
    feat = numpy.log(feat)
    feat = dct(feat, type=2, axis=1, norm='ortho')[:,:numcep]
    feat = lifter(feat,ceplifter)
    if appendEnergy: feat[:,0] = numpy.log(energy) # replace first cepstral coefficient with log of frame energy
    return feat

def fbank(signal,samplerate=16000,winlen=0.025,winstep=0.01,
          nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,
          winfunc=lambda x:numpy.ones((x,))):
    """Compute Mel-filterbank energy features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is 512.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: 2 values. The first is a numpy array of size (NUMFRAMES by nfilt) containing features. Each row holds 1 feature vector. The
        second return value is the energy in each frame (total energy, unwindowed)
    """
    highfreq= highfreq or samplerate/2
    signal = sigproc.preemphasis(signal,preemph)
    frames = sigproc.framesig(signal, winlen*samplerate, winstep*samplerate, winfunc)
    pspec = sigproc.powspec(frames,nfft)
    energy = numpy.sum(pspec,1) # this stores the total energy in each frame
    energy = numpy.where(energy == 0,numpy.finfo(float).eps,energy) # if energy is zero, we get problems with log

    fb = get_filterbanks(nfilt,nfft,samplerate,lowfreq,highfreq)
    feat = numpy.dot(pspec,fb.T) # compute the filterbank energies
    feat = numpy.where(feat == 0,numpy.finfo(float).eps,feat) # if feat is zero, we get problems with log

    return feat,energy

def logfbank(signal,samplerate=16000,winlen=0.025,winstep=0.01,
             nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,
             winfunc=lambda x:numpy.ones((x,))):
    """Compute log Mel-filterbank energy features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is 512.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: A numpy array of size (NUMFRAMES by nfilt) containing features. Each row holds 1 feature vector.
    """
    feat,energy = fbank(signal,samplerate,winlen,winstep,nfilt,nfft,lowfreq,highfreq,preemph,winfunc)
    return numpy.log(feat)

def ssc(signal,samplerate=16000,winlen=0.025,winstep=0.01,
        nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,
        winfunc=lambda x:numpy.ones((x,))):
    """Compute Spectral Subband Centroid features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is 512.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: A numpy array of size (NUMFRAMES by nfilt) containing features. Each row holds 1 feature vector.
    """
    highfreq= highfreq or samplerate/2
    signal = sigproc.preemphasis(signal,preemph)
    frames = sigproc.framesig(signal, winlen*samplerate, winstep*samplerate, winfunc)
    pspec = sigproc.powspec(frames,nfft)
    pspec = numpy.where(pspec == 0,numpy.finfo(float).eps,pspec) # if things are all zeros we get problems

    fb = get_filterbanks(nfilt,nfft,samplerate,lowfreq,highfreq)
    feat = numpy.dot(pspec,fb.T) # compute the filterbank energies
    R = numpy.tile(numpy.linspace(1,samplerate/2,numpy.size(pspec,1)),(numpy.size(pspec,0),1))

    return numpy.dot(pspec*R,fb.T) / feat

def hz2mel(hz):
    """Convert a value in Hertz to Mels
    :param hz: a value in Hz. This can also be a numpy array, conversion proceeds element-wise.
    :returns: a value in Mels. If an array was passed in, an identical sized array is returned.
    """
    return 2595 * numpy.log10(1+hz/700.)

def mel2hz(mel):
    """Convert a value in Mels to Hertz
    :param mel: a value in Mels. This can also be a numpy array, conversion proceeds element-wise.
    :returns: a value in Hertz. If an array was passed in, an identical sized array is returned.
    """
    return 700*(10**(mel/2595.0)-1)

def get_filterbanks(nfilt=20,nfft=512,samplerate=16000,lowfreq=0,highfreq=None):
    """Compute a Mel-filterbank. The filters are stored in the rows, the columns correspond
    to fft bins. The filters are returned as an array of size nfilt * (nfft/2 + 1)
    :param nfilt: the number of filters in the filterbank, default 20.
    :param nfft: the FFT size. Default is 512.
    :param samplerate: the sample rate of the signal we are working with, in Hz. Affects mel spacing.
    :param lowfreq: lowest band edge of mel filters, default 0 Hz
    :param highfreq: highest band edge of mel filters, default samplerate/2
    :returns: A numpy array of size nfilt * (nfft/2 + 1) containing filterbank. Each row holds 1 filter.
    """
    highfreq= highfreq or samplerate/2
    assert highfreq <= samplerate/2, "highfreq is greater than samplerate/2"

    # compute points evenly spaced in mels
    lowmel = hz2mel(lowfreq)
    highmel = hz2mel(highfreq)
    melpoints = numpy.linspace(lowmel,highmel,nfilt+2)
    # our points are in Hz, but we use fft bins, so we have to convert
    #  from Hz to fft bin number
    bin = numpy.floor((nfft+1)*mel2hz(melpoints)/samplerate)

    fbank = numpy.zeros([nfilt,nfft//2+1])
    for j in range(0,nfilt):
        for i in range(int(bin[j]), int(bin[j+1])):
            fbank[j,i] = (i - bin[j]) / (bin[j+1]-bin[j])
        for i in range(int(bin[j+1]), int(bin[j+2])):
            fbank[j,i] = (bin[j+2]-i) / (bin[j+2]-bin[j+1])
    return fbank

def lifter(cepstra, L=22):
    """Apply a cepstral lifter the the matrix of cepstra. This has the effect of increasing the
    magnitude of the high frequency DCT coeffs.
    :param cepstra: the matrix of mel-cepstra, will be numframes * numcep in size.
    :param L: the liftering coefficient to use. Default is 22. L <= 0 disables lifter.
    """
    if L > 0:
        nframes,ncoeff = numpy.shape(cepstra)
        n = numpy.arange(ncoeff)
        lift = 1 + (L/2.)*numpy.sin(numpy.pi*n/L)
        return lift*cepstra
    else:
        # values of L <= 0, do nothing
        return cepstra

def delta(feat, N):
    """Compute delta features from a feature vector sequence.
    :param feat: A numpy array of size (NUMFRAMES by number of features) containing features. Each row holds 1 feature vector.
    :param N: For each frame, calculate delta features based on preceding and following N frames
    :returns: A numpy array of size (NUMFRAMES by number of features) containing delta features. Each row holds 1 delta feature vector.
    """
    if N < 1:
        raise ValueError('N must be an integer >= 1')
    NUMFRAMES = len(feat)
    denominator = 2 * sum([i**2 for i in range(1, N+1)])
    delta_feat = numpy.empty_like(feat)
    padded = numpy.pad(feat, ((N, N), (0, 0)), mode='edge')   # padded version of feat
    for t in range(NUMFRAMES):
        delta_feat[t] = numpy.dot(numpy.arange(-N, N+1), padded[t : t+2*N+1]) / denominator   # [t : t+2*N+1] == [(N+t)-N : (N+t)+N+1]
    return delta_feat